# load data

In [1]:
custom_newsfeedcustom_newsfeed

In [2]:
cnx = sqlite3.connect("../home.db")
df_tweets = pd.read_sql_query("SELECT id, full_text FROM tweets", cnx)
df_tweets_2 = df_tweets.copy()

In [3]:
with open('news_domains.txt', 'r') as f:
    news_domains = json.loads(f.read())

# utils

In [4]:
def find_url(tweet):
    return re.findall("http\S+", tweet)

def clean_links(tweet):
    tweet = re.sub(r'bit.ly/\S+', '', tweet)
    tweet = re.sub(r't.co/\S+', '', tweet)
    tweet = re.sub(r'buff.ly/\S+', '', tweet)
    tweet = re.sub(r'twitter.com/\S+', '', tweet)
    return tweet

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain
    
def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return(l)

# function for df

In [5]:
def find_news(df):

    df["urls"] = df["full_text"].apply(find_url)
    df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)


    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())


    for col in new_columns_list:
        df[col] = df[col].isin(news_domains)

    df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)
    
    return df


In [6]:
%%time
df_tweets = find_news(df_tweets)

CPU times: user 569 ms, sys: 7.93 ms, total: 577 ms
Wall time: 577 ms


In [8]:
df_tweets[df_tweets["contains_news"] > 0]

,id,full_text,contains_news
96,986941808348409856,My comments in The Guardian on implementation ...,1
150,1083458327143739392,Cyber insurance experts agree they're not sure...,1
188,1121030235108118528,@wmarybeard @Lord_Keynes2 @Steve_Sailer The Yo...,1
196,1129155364404858880,Peasants in medieval England ate a diet of mea...,1
222,1153145699086680069,The Finns know the score: regular sauna usage ...,1
...,...,...,...
50173,1350121300925313028,"Individual contributions dominate in Canada, a...",1
50203,1350126505339346957,A Webflow founder posts how Hackernews saved t...,1
50215,1350129672529481728,"Sources: at an all-hands staff meeting, incomi...",1
50216,1350129720311115777,"In 10 years: In order to train our new model, ...",1


# function for single tweet

In [9]:
tweet1 = df_tweets_2.iloc[0]
print(tweet1.full_text)

I wish I had kept my 1,700 BTC @ $0.06 instead of selling them at $0.30, now that they're $8.00! #bitcoin


In [10]:
def find_news(tweet_full_text):
    urls = find_url(tweet_full_text)
    urls = [clean_links(d) for d in urls]
    domains = [get_domain(d) for d in urls]
    domains = remove_empty_str(domains)
    contains_news = 0
    for i in domains:
        if i in news_domains:
            contains_news = 1
            break
            
    return contains_news

In [11]:
find_news(tweet1.full_text) is True

False

In [13]:
%%time
df_tweets_2["contains_news"] = df_tweets_2.full_text.apply(find_news)

CPU times: user 1.95 s, sys: 0 ns, total: 1.95 s
Wall time: 1.95 s


In [14]:
df_tweets_2[df_tweets_2["contains_news"] > 0]

,id,full_text,contains_news
96,986941808348409856,My comments in The Guardian on implementation ...,1
150,1083458327143739392,Cyber insurance experts agree they're not sure...,1
188,1121030235108118528,@wmarybeard @Lord_Keynes2 @Steve_Sailer The Yo...,1
196,1129155364404858880,Peasants in medieval England ate a diet of mea...,1
222,1153145699086680069,The Finns know the score: regular sauna usage ...,1
...,...,...,...
50173,1350121300925313028,"Individual contributions dominate in Canada, a...",1
50203,1350126505339346957,A Webflow founder posts how Hackernews saved t...,1
50215,1350129672529481728,"Sources: at an all-hands staff meeting, incomi...",1
50216,1350129720311115777,"In 10 years: In order to train our new model, ...",1


In [15]:
df_tweets[df_tweets["contains_news"] != df_tweets_2["contains_news"]]

,id,full_text,contains_news
